In [ ]:
%load_ext autoreload
%autoreload 2

Please keep this notebook functionally at par with scripts/oncall/generate_report.py AND linearly runnable (no code with duplicated functionality; run all from this cell down should produce a report). Modify the parameters in `single_fitting_cycle` and `plot_trials` per customizations

In [ ]:
import numpy as np
import pandas as pd

import datetime

import copy
import json
import time
import os

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIRHD

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.sensitivity import calculate_sensitivity_and_plot
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, scale_up_testing_and_forecast
from main.seir.forecast import create_decile_csv
from main.seir.uncertainty import MCUncertainty
from utils.generic.create_report import save_dict_and_create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

## Params

In [ ]:
output_folder = '../../reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

model=SEIRHD
forecast_days = 45

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)

num_trials_to_plot = 10
plot_topk_trials_for_columns = [Columns.confirmed, Columns.active]
plot_ptiles_for_columns = [Columns.confirmed, Columns.active]
sort_trials_by_column = Columns.confirmed
date_of_sorting_trials = '2020-08-31'

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, train_period=21, val_period=7, num_evals=1000,
    data_from_tracker=False, model=model, 
    smooth_jump=True,
    which_compartments=['active', 'total', 'deceased', 'recovered'])
predictions_dict['m2'] = single_fitting_cycle(
    state, district, train_period=21, val_period=0, num_evals=1000,
    data_from_tracker=False, model=model, 
    smooth_jump=True,
    which_compartments=['active', 'total', 'deceased', 'recovered'])

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m2']['data_last_date']

## Fitting Loss

### M1 Loss

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss

In [ ]:
predictions_dict['m2']['df_loss']

## Sensitivity Plots

In [ ]:
predictions_dict['m1']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m1')
predictions_dict['m2']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m2')

## Plot Forecasts

In [ ]:
predictions_dict['m2']['forecasts'] = {}
predictions_dict['m2']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m2', model=model, days=forecast_days)
predictions_dict['m2']['plots']['forecast_best'] = plot_forecast(predictions_dict, (state, district), error_bars=True)

trials_processed = forecast_all_trials(predictions_dict, train_fit='m1', model=model, forecast_days=forecast_days)
predictions_dict['m1']['trials_processed'] = trials_processed
predictions_dict['m1']['all_trials'] = trials_to_df(trials_processed, column=sort_trials_by_column)
trials_processed = forecast_all_trials(predictions_dict, train_fit='m2', model=model, forecast_days=forecast_days)
predictions_dict['m2']['trials_processed'] = trials_processed
predictions_dict['m2']['all_trials'] = trials_to_df(trials_processed, column=sort_trials_by_column)
kforecasts = plot_top_k_trials(
    predictions_dict,
    train_fit='m2',
    trials_processed=predictions_dict['m2']['trials_processed'], 
    k=num_trials_to_plot,
    which_compartments=plot_topk_trials_for_columns)
predictions_dict['m2']['plots']['forecasts_topk'] = {}
for column in plot_topk_trials_for_columns:
    predictions_dict['m2']['plots']['forecasts_topk'][column.name] = kforecasts[column]

## Uncertainty

In [ ]:
uncertainty = MCUncertainty(predictions_dict, date_of_sorting_trials)

In [ ]:
uncertainty.beta_loss

## Uncertainty Forecasts

In [ ]:
uncertainty_forecasts = uncertainty.get_forecasts()
for key in uncertainty_forecasts.keys():
    predictions_dict['m2']['forecasts'][key] = uncertainty_forecasts[key]['df_prediction']

In [ ]:
predictions_dict['m2']['beta'] = uncertainty.beta
predictions_dict['m2']['beta_loss'] = uncertainty.beta_loss

In [ ]:
predictions_dict['m2']['deciles'] = uncertainty_forecasts

In [ ]:
predictions_dict['m2']['plots']['forecast_best_50'] = plot_forecast(predictions_dict, (state, district), fits_to_plot=['best', 50], error_bars=False)
predictions_dict['m2']['plots']['forecast_best_80'] = plot_forecast(predictions_dict, (state, district), fits_to_plot=['best', 80], error_bars=False)

In [ ]:
ptiles_plots = plot_ptiles(predictions_dict, which_compartments=plot_ptiles_for_columns)
predictions_dict['m2']['plots']['forecasts_ptiles'] = {}
for column in plot_ptiles_for_columns:
    predictions_dict['m2']['plots']['forecasts_ptiles'][column.name] = ptiles_plots[column]

## Create Report

In [ ]:
save_dict_and_create_report(predictions_dict, ROOT_DIR=output_folder)

In [ ]:
create_decile_csv(predictions_dict, 'Mumbai', 'district')